In [1]:
%load_ext autoreload
%autoreload 2
import json 
import pysftp
import os
import pandas_gbq
import pandas as pd
from modules.buckets import *
from modules.reproducibility import *
import logging

#Configure loggging
logging.basicConfig(filename='BigQuery.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New Big Query Logging Instance')

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact, import the SFTP password from the same file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'powerschool-420113-db919282054b.json'
with open('powerschool-420113-db919282054b.json') as json_file:
    j = json.load(json_file) 
    sftp_pass = j['sftp_password']
    
# ----------------------------------------------------------
#Need to make this portion to where it assesses all files in the dir recursively. 
def main(SFTP_folder_name):

    SFTP_folder_name  = initial_schema_check(SFTP_folder_name)
    print(SFTP_folder_name)

    instance = Create(
                project_id='powerschool-420113',
                location = 'us-south1',
                bucket=f'{SFTP_folder_name}bucket-iotaschools-1',
                local_dir = fr'S:\SFTP\{SFTP_folder_name}',
                db = SFTP_folder_name,
                append_or_replace='replace',
                )
    
    instance.process()# Pass SFTP files into Bucket & then into Big Query tables



main("powerschool")
main("EIS")

powerschool


Bucket powerschoolbucket-iotaschools-1 already exists.
Local File S:\SFTP\powerschool\Students_export.txt was uploaded as a new file Students_export.txt in the bucket powerschoolbucket-iotaschools-1.
Local File S:\SFTP\powerschool\Student_Records.csv was uploaded as a new file Student_Records.csv in the bucket powerschoolbucket-iotaschools-1.
powerschoolbucket-iotaschools-1
Attempting to create table powerschool-420113.powerschool.Student_Records and send data for the first time
Attempting to create table powerschool-420113.powerschool.Students_export and send data for the first time
eis


Bucket eisbucket-iotaschools-1 already exists.
Local File S:\SFTP\eis\EIS_prior_schools.csv was uploaded as a new file EIS_prior_schools.csv in the bucket eisbucket-iotaschools-1.
eisbucket-iotaschools-1
Attempting to create table powerschool-420113.eis.EIS_prior_schools and send data for the first time


In [6]:
SFTP_folder_name = 'powerschool'
local_dir = fr'S:\SFTP\{SFTP_folder_name}'

for filename in os.listdir(local_dir):
    local_file_path = if os.path.isfile(os.path.join(local_dir, filename)):
    upload_to_bucket(filename, local_file_path, bucket_name)


# def upload_all_files_to_bucket(local_dir, bucket_name):

#     for filename in os.listdir(local_dir):
#         # if os.path.isfile(os.path.join(filename)):  #'S:\\SFTP\\\\powerschool'
#             upload_to_bucket(filename, os.path.join(filename), bucket_name)

Students_export.txt
Student_Records.csv
